In [ ]:
import numpy as np
import pandas as pd
from requests import get

from datetime import timedelta
from time import sleep, time
import plotly.graph_objects as go
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})
from dash import Dash
from dash_bootstrap_templates import load_figure_template
from jupyter_dash import JupyterDash
from IPython import get_ipython
import dash_bootstrap_components as dbc

load_figure_template("darkly")
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
try:
    shell = str(get_ipython())
    if "ZMQInteractiveShell" in shell:
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    elif shell == "TerminalInteractiveShell":
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    else:
        app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
except NameError:
    app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])

bg_color = "#0b0b18"

%load_ext autoreload
%autoreload 2

In [ ]:
candles = dl_ex_candles(exchange="binance_us", symbol="BTCUSDT", timeframe="1h", candles_to_dl=200)
closes = candles[:, 4]

In [ ]:
def wma_tv(source: np.array, length: int = 10):
    weight = np.flip((length - np.arange(0, length)) * length)
    norm = weight.sum()

    wma = np.full_like(source, np.nan)
    len_minus_one = length - 1

    for index in range(len_minus_one, source.size):
        the_sum = (source[index - len_minus_one : index + 1] * weight).sum()
        wma[index] = the_sum / norm

    return wma

In [ ]:
wma = wma_tv(source=closes, length=14)

In [ ]:
def plot_wma(
    candles: np.array,
    indicator: np.array,
    ind_color: str = "#3EA3FF",
):
    datetimes = pd.to_datetime(candles[:, 0], unit="ms")
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=datetimes,
                open=candles[:, 1],
                high=candles[:, 2],
                low=candles[:, 3],
                close=candles[:, 4],
                name="Candles",
            ),
            go.Scatter(
                x=datetimes,
                y=indicator,
                name="WMA",
                line_color=ind_color,
            ),
        ]
    )
    fig.update_layout(height=500, xaxis_rangeslider_visible=False)
    fig.show()

In [ ]:
plot_wma(candles=candles, indicator=wma)